# EFM Weak Force Analogue Simulation (T/S State, N=400, Linear Damping)

This notebook simulates the Ehokolo Fluxon Model (EFM) NLKG equation within the hypothesized Time/Space (T/S) state parameters, replacing the previous dissipation term with a **linear damping term (-gamma * dphi/dt)** to investigate dynamics analogous to particle decay or transformation.

**Objectives**
- Implement the EFM NLKG solver with T/S state parameters (reduced c_eff) and linear damping.
- Initialize the field with a stable background density (n'=2) plus an unstable perturbation (Gaussian bump).
- Simulate the evolution on an N=400 grid.
- Monitor the stability, energy (should decrease due to damping), and evolution of the perturbation.
- Analyze if linear damping leads to decay/relaxation dynamics.

## 1. Setup: Libraries, GPU Check, Drive Mount, Paths

In [ ]:
# Clear GPU memory at the start
import torch
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

# Install/Import required libraries
# !pip install torch numpy matplotlib tqdm psutil scipy -q
!nvidia-smi

import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import psutil
import time
from datetime import datetime
from google.colab import drive
import os
import gc

# Check GPU and memory
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"System RAM Total: {psutil.virtual_memory().total / 1e9:.2f} GB")

# Mount Google Drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/EFM_Simulations/'
wf_path = os.path.join(base_path, 'WeakForce_Analogue_N400_LinearDamp/') # Specific folder
checkpoint_path = os.path.join(wf_path, 'checkpoints/')
data_path = os.path.join(wf_path, 'data/')
plot_path = os.path.join(wf_path, 'plots/')
os.makedirs(checkpoint_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)
print(f"Paths created/checked:\n Checkpoints: {checkpoint_path}\n Data: {data_path}\n Plots: {plot_path}")

## 2. Simulation Parameters (T/S State, N=400, Linear Damping)

In [ ]:
# --- Numerical Parameters ---
N = 400  # Grid size 
L = 10.0  # Box size (simulation units)
dx = L / N
dt = 0.00005 # Time step (Keep small for stability)
T_steps = 10000 # Total steps (Adjust based on decay timescale)
save_interval = 100 # How often to save metrics/plots 
checkpoint_interval = 1000 

# --- Physical Parameters (Baseline NLKG + T/S State + Linear Damping) ---
c_eff_sq = 0.1 # Hypothesis: Reduced c^2 for T/S state (c=1 baseline)
c_eff = np.sqrt(c_eff_sq)
m2 = 1.0     # Mass term squared 
g = 0.1     # Cubic nonlinearity coefficient 
eta = 0.01    # Quintic nonlinearity coefficient 
k_rho = 0.01  # Density coupling constant (rho = k * phi^2)
alpha = 0.1   # State parameter (Keep as before for now)
gamma_damp = 0.1 # <<<<<----- LINEAR DAMPING COEFFICIENT ----->>>>>

# --- HDS Parameters ---
rho_ref = 1.5 # Reference density from HDS validation
n_prime_background = 2.0 # Target background density level for T/S state
target_rho = rho_ref / n_prime_background

# --- Initial Conditions --- 
background_amp = np.sqrt(target_rho / k_rho) if k_rho > 0 else 1.0
noise_level = 0.001 # Very small background noise
bump_amplitude = 1.0  # Amplitude of the 'unstable particle'
bump_width = 0.5     # Spatial width of the 'particle'

# --- Precision ---
dtype = torch.float16 # Use float16 for memory

# --- Reporting ---
print("--- T/S Weak Force Analogue Parameters (Linear Damping) ---")
print(f"Grid Size (N): {N}^3")
print(f"Time Step (dt): {dt}")
print(f"Total Steps (T_steps): {T_steps}")
print(f"Effective c^2 (c_eff_sq): {c_eff_sq}")
print(f"Mass Term (m^2): {m2}")
print(f"Cubic Term (g): {g}")
print(f"Quintic Term (eta): {eta}")
print(f"Alpha (α): {alpha}")
print(f"Gamma Damping (γ): {gamma_damp}")
print(f"Background n': {n_prime_background}")
print(f"Target Background ρ: {target_rho:.4f}")
print(f"Precision (dtype): {dtype}")
print("--------------------------------------------------------")

## 3. Helper Functions (Includes Linear Damping)

In [6]:
# Potential function V(phi) = 0.5*m2*phi^2 - 0.25*g*phi^4 + 0.1667*eta*phi^6
def potential(phi, m2_p, g_p, eta_p):
    phi_f32 = phi.to(torch.float32) 
    m2_p_f32 = torch.tensor(m2_p, dtype=torch.float32, device=phi.device)
    g_p_f32 = torch.tensor(g_p, dtype=torch.float32, device=phi.device)
    eta_p_f32 = torch.tensor(eta_p, dtype=torch.float32, device=phi.device)
    term2 = 0.5 * m2_p_f32 * phi_f32**2
    term4 = -0.25 * g_p_f32 * phi_f32**4
    term6 = (1.0/6.0) * eta_p_f32 * phi_f32**6
    return (term2 + term4 + term6).to(phi.dtype)

# NLKG derivative calculation including linear damping (-gamma * phi_dot)
# Rearranged: ∂²ϕ/∂t² = c²∇²ϕ - V'(φ) - γ(∂φ/∂t)
def nlkg_derivative_wf_linear_damp(phi, phi_dot, m2_p, g_p, eta_p, c_eff_sq_p, alpha_p, gamma_damp_p, L_p, N_p, dx_p, device_p):
    phi_f32 = phi.to(torch.float32)
    phi_dot_f32 = phi_dot.to(torch.float32)
    dx_p_f32 = torch.tensor(dx_p, dtype=torch.float32, device=device_p)
    
    # Calculate Laplacian 
    laplacian_f32 = torch.zeros_like(phi_f32)
    for dim in range(3):
        laplacian_f32 += torch.roll(phi_f32, shifts=1, dims=dim)
        laplacian_f32 += torch.roll(phi_f32, shifts=-1, dims=dim)
    laplacian = (laplacian_f32 - 6.0 * phi_f32) / dx_p_f32**2

    # Calculate V'(phi)
    m2_p_f32 = torch.tensor(m2_p, dtype=torch.float32, device=device_p)
    g_p_f32 = torch.tensor(g_p, dtype=torch.float32, device=device_p)
    eta_p_f32 = torch.tensor(eta_p, dtype=torch.float32, device=device_p)
    dV_dphi = m2_p_f32 * phi_f32 - g_p_f32 * phi_f32**3 + eta_p_f32 * phi_f32**5
    
    # Alpha term (Omitted for now)
    alpha_term = torch.zeros_like(phi_f32) 
        
    # Linear Damping term 
    linear_damping_term = torch.zeros_like(phi_f32)
    if gamma_damp_p != 0:
        gamma_damp_p_f32 = torch.tensor(gamma_damp_p, dtype=torch.float32, device=device_p)
        linear_damping_term = -gamma_damp_p_f32 * phi_dot_f32 # Damping opposes velocity
    
    # Calculate phi_ddot 
    c_eff_sq_p_f32 = torch.tensor(c_eff_sq_p, dtype=torch.float32, device=device_p)
    phi_ddot = c_eff_sq_p_f32 * laplacian - dV_dphi + alpha_term + linear_damping_term 
    
    # Apply absorbing boundary conditions (damping mask to phi_dot and phi_ddot)
    boundary_width = int(0.1 * N_p) 
    damping_factor = 0.05 
    mask = torch.ones_like(phi)
    for dim in range(3):
        indices = torch.arange(N_p, device=device_p, dtype=phi.dtype)
        damping_profile = torch.ones(N_p, device=device_p, dtype=phi.dtype)
        ramp = torch.linspace(1.0, damping_factor, boundary_width, device=device_p, dtype=phi.dtype)
        damping_profile[:boundary_width] = ramp.flip(dims=[0])
        damping_profile[-boundary_width:] = ramp
        if dim == 0: mask *= damping_profile[:, None, None]
        elif dim == 1: mask *= damping_profile[None, :, None]
        else: mask *= damping_profile[None, None, :]
            
    phi_dot_damped = phi_dot_f32 * mask.to(torch.float32)
    phi_ddot_damped = phi_ddot * mask.to(torch.float32) 

    return phi_dot_damped.to(phi_dot.dtype), phi_ddot_damped.to(phi_ddot.dtype)

# RK4 integrator - calls the correct derivative function
def update_phi_rk4_wf(phi, phi_dot, dt_p, m2_p, g_p, eta_p, c_eff_sq_p, alpha_p, gamma_damp_p, L_p, N_p, dx_p, device_p):
    with torch.no_grad():
        original_dtype = phi.dtype
        phi_f32 = phi.to(torch.float32)
        phi_dot_f32 = phi_dot.to(torch.float32)
        dt_p_f32 = torch.tensor(dt_p, dtype=torch.float32, device=device_p)
        
        k1_v, k1_a = nlkg_derivative_wf_linear_damp(phi_f32, phi_dot_f32, m2_p, g_p, eta_p, c_eff_sq_p, alpha_p, gamma_damp_p, L_p, N_p, dx_p, device_p)
        k1_v=k1_v.to(torch.float32); k1_a=k1_a.to(torch.float32)

        k2_v, k2_a = nlkg_derivative_wf_linear_damp(phi_f32 + 0.5 * dt_p_f32 * k1_v, phi_dot_f32 + 0.5 * dt_p_f32 * k1_a, m2_p, g_p, eta_p, c_eff_sq_p, alpha_p, gamma_damp_p, L_p, N_p, dx_p, device_p)
        k2_v=k2_v.to(torch.float32); k2_a=k2_a.to(torch.float32)

        k3_v, k3_a = nlkg_derivative_wf_linear_damp(phi_f32 + 0.5 * dt_p_f32 * k2_v, phi_dot_f32 + 0.5 * dt_p_f32 * k2_a, m2_p, g_p, eta_p, c_eff_sq_p, alpha_p, gamma_damp_p, L_p, N_p, dx_p, device_p)
        k3_v=k3_v.to(torch.float32); k3_a=k3_a.to(torch.float32)

        k4_v, k4_a = nlkg_derivative_wf_linear_damp(phi_f32 + dt_p_f32 * k3_v, phi_dot_f32 + dt_p_f32 * k3_a, m2_p, g_p, eta_p, c_eff_sq_p, alpha_p, gamma_damp_p, L_p, N_p, dx_p, device_p)
        k4_v=k4_v.to(torch.float32); k4_a=k4_a.to(torch.float32)
        
        phi_new_f32 = phi_f32 + (dt_p_f32 / 6.0) * (k1_v + 2.0 * k2_v + 2.0 * k3_v + k4_v)
        phi_dot_new_f32 = phi_dot_f32 + (dt_p_f32 / 6.0) * (k1_a + 2.0 * k2_a + 2.0 * k3_a + k4_a)

        phi_new = phi_new_f32.to(original_dtype)
        phi_dot_new = phi_dot_new_f32.to(original_dtype)

        del phi_f32, phi_dot_f32, k1_v, k1_a, k2_v, k2_a, k3_v, k3_a, k4_v, k4_a 
        if torch.cuda.is_available(): torch.cuda.empty_cache()

        return phi_new, phi_dot_new

# Function to compute metrics (Energy requires c_eff, not c_eff_sq)
def compute_metrics_wf(phi, phi_dot, k_rho_p, m2_p, g_p, eta_p, c_eff_p, dx_p, device_p):
    # Reuse the metric function from HDS notebook - energy calc uses c_eff correctly
    with torch.no_grad():
        phi_f32 = phi.to(torch.float32)
        phi_dot_f32 = phi_dot.to(torch.float32)
        dx_p_f32 = torch.tensor(dx_p, dtype=torch.float32, device=device_p)
        c_eff_p_f32 = torch.tensor(c_eff_p, dtype=torch.float32, device=device_p)

        max_amp = torch.max(torch.abs(phi_f32)).item()
        avg_density = k_rho_p * torch.mean(phi_f32**2).item()

        kinetic_energy_density = 0.5 * phi_dot_f32**2
        grad_phi_tuple = torch.gradient(phi_f32, spacing=dx_p_f32, dim=[0, 1, 2])
        gradient_energy_density = 0.5 * c_eff_p_f32**2 * (grad_phi_tuple[0]**2 + grad_phi_tuple[1]**2 + grad_phi_tuple[2]**2)
        potential_energy_density = potential(phi_f32, m2_p, g_p, eta_p)
        total_energy_density = kinetic_energy_density + gradient_energy_density + potential_energy_density
        total_energy = torch.sum(total_energy_density).item() * (dx_p_f32.item()**3)

        if not (np.isfinite(max_amp) and np.isfinite(avg_density) and np.isfinite(total_energy)):
             return float('nan'), float('nan'), float('nan')
        
        del phi_f32, phi_dot_f32, kinetic_energy_density, grad_phi_tuple, gradient_energy_density, potential_energy_density, total_energy_density
        if torch.cuda.is_available(): torch.cuda.empty_cache()

        return max_amp, avg_density, total_energy

## 4. Simulation Execution (T/S State with Linear Damping)

In [ ]:
print(f"\n--- Starting T/S State Simulation with Linear Damping --- (N={N})")
sim_start_time = time.time()

# Retrieve parameters 
n_prime_to_run = n_prime_background # Should be 2.0

# Calculate target density and initial field amplitude
target_rho = rho_ref / n_prime_to_run
initial_amp_bg = np.sqrt(target_rho / k_rho) if k_rho > 0 else 1.0 
print(f" Target Background Density: {target_rho:.4f}, Background Amplitude: {initial_amp_bg:.4f}")
print(f" Initial Bump Amp: {bump_amplitude}, Width: {bump_width}")
print(f" Linear Damping γ: {gamma_damp}")

# Initialize fields on GPU 
print("Initializing fields...")
x_coord = torch.linspace(-L/2, L/2, N, device=device, dtype=dtype)
X, Y, Z = torch.meshgrid(x_coord, x_coord, x_coord, indexing='ij')
R_center = torch.sqrt(X**2 + Y**2 + Z**2)

phi_bg = (torch.ones((N, N, N), device=device, dtype=dtype) * initial_amp_bg * 
          (1 + noise_level * torch.randn((N, N, N), device=device, dtype=dtype)))
phi_bump = bump_amplitude * torch.exp(-R_center**2 / bump_width**2)
phi = phi_bg + phi_bump.to(dtype) 
phi_dot = torch.zeros_like(phi)

print(f"Field initialization complete. Initial Max|φ|: {torch.max(torch.abs(phi)):.2f}")
initial_max_amp, initial_avg_rho, initial_energy = compute_metrics_wf(
    phi, phi_dot, k_rho, m2, g, eta, c_eff, dx, device)
print(f"Initial Metrics: Max|φ|={initial_max_amp:.2f}, <ρ>={initial_avg_rho:.4f}, E={initial_energy:.2e}")

del phi_bg, phi_bump, X, Y, Z, R_center, x_coord
if torch.cuda.is_available(): torch.cuda.empty_cache()
gc.collect()

# Track metrics
run_max_amp_history = [initial_max_amp]
run_avg_rho_history = [initial_avg_rho]
run_energy_history = [initial_energy]
run_stable = True
run_steps_completed = 0

# --- Simulation Loop ---
pbar = tqdm(range(T_steps), desc=f"T/S State (n'=2, γ={gamma_damp})")
for t in pbar:
    try:
        # Use the WF updater with linear damping derivative
        phi, phi_dot = update_phi_rk4_wf(phi, phi_dot, dt, m2, g, eta, c_eff_sq, alpha, gamma_damp, L, N, dx, device)
        run_steps_completed = t + 1
    except RuntimeError as e:
         if "out of memory" in str(e):
             print(f"\n!!! CUDA out of memory at step {t}. Stopping run. !!!")
             run_stable = False; break
         else:
             print(f"\n!!! Runtime error at step {t}: {e} !!!"); run_stable = False; break
    except Exception as e:
        print(f"\n!!! Unexpected error at step {t}: {e} !!!"); run_stable = False; break
        
    if t % save_interval == 0 or t == T_steps - 1:
         max_amp, avg_density, total_energy = compute_metrics_wf(phi, phi_dot, k_rho, m2, g, eta, c_eff, dx, device)
         if not (np.isfinite(max_amp) and np.isfinite(avg_density) and np.isfinite(total_energy)) or max_amp > 50.0: 
             print(f"\n!!! Instability detected at step {t} (Max Amp: {max_amp:.2e}, Avg Rho: {avg_density:.4f}, Energy: {total_energy:.2e}) !!!")
             run_stable = False
             run_max_amp_history.append(max_amp if np.isfinite(max_amp) else float('nan'))
             run_avg_rho_history.append(avg_density if np.isfinite(avg_density) else float('nan'))
             run_energy_history.append(total_energy if np.isfinite(total_energy) else float('nan'))
             break
         else:
            run_max_amp_history.append(max_amp); run_avg_rho_history.append(avg_density); run_energy_history.append(total_energy)
            pbar.set_postfix({'Max|φ|': f'{max_amp:.2f}', '<ρ>': f'{avg_density:.4f}', 'E': f'{total_energy:.2e}'})
            
    if t % checkpoint_interval == 0 and t > 0:
        try:
            chkpt_file = os.path.join(checkpoint_path, f"wf_analogue_lin_damp_step{t}.pt")
            torch.save({'step': t, 'phi': phi.cpu(), 'phi_dot': phi_dot.cpu()}, chkpt_file)
            phi = phi.to(device); phi_dot = phi_dot.to(device)
        except Exception as e:
            print(f"Warning: Could not save checkpoint at step {t}: {e}")

pbar.close()

final_phi_slice_np = None
if run_stable and 'phi' in locals() and phi is not None: 
    try: final_phi_slice_np = phi[N//2, :, :].cpu().numpy().astype(np.float16) 
    except Exception as e: print(f"Error converting final phi slice: {e}"); final_phi_slice_np = None

final_results_wf = {
    'n_prime': n_prime_background, 'stable': run_stable, 'steps_completed': run_steps_completed,
    'max_amp': np.array(run_max_amp_history, dtype=np.float32), 
    'avg_rho': np.array(run_avg_rho_history, dtype=np.float32),
    'energy': np.array(run_energy_history, dtype=np.float32),
    'final_phi_slice': final_phi_slice_np
}

try:
    npz_file_path = os.path.join(data_path, f"wf_analogue_lin_damp_results_g{gamma_damp:.2f}.npz") # Include gamma in filename
    np.savez(npz_file_path, **final_results_wf)
    print(f"Saved final results for T/S state (gamma={gamma_damp}) to {npz_file_path}")
except Exception as e: print(f"Error saving final results: {e}")

if 'phi' in locals(): del phi 
if 'phi_dot' in locals(): del phi_dot 
del run_max_amp_history, run_avg_rho_history, run_energy_history
if 'final_phi_slice_np' in locals(): del final_phi_slice_np
if torch.cuda.is_available(): torch.cuda.empty_cache()
gc.collect()

sim_end_time = time.time()
print(f"--- T/S State Simulation (Linear Damping γ={gamma_damp}) finished in {(sim_end_time - sim_start_time) / 60:.2f} minutes ---")

## 5. Analysis and Visualization (T/S State with Linear Damping)

In [ ]:
# --- Analysis & Visualization for T/S Run with Linear Damping ---
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import Image, display

# --- Ensure parameters are loaded/available from the simulation cell ---
# If running analysis in a separate session, redefine necessary params:
# N = 400; L = 10.0; dx = L/N; 
# save_interval = 100 # Match the interval used in simulation
# rho_ref = 1.5; n_prime_background = 2.0; target_rho = rho_ref / n_prime_background
# gamma_damp = 0.1 # Match the gamma used in simulation
# data_path = '/content/drive/MyDrive/EFM_Simulations/WeakForce_Analogue_N400_LinearDamp/data/'
# plot_path = '/content/drive/MyDrive/EFM_Simulations/WeakForce_Analogue_N400_LinearDamp/plots/'
# ---------------------------------------------------------------------

# Load the saved T/S results with linear damping
n_prime_bg = n_prime_background 
gamma_val = gamma_damp # Use the parameter value
results_file = os.path.join(data_path, f"wf_analogue_lin_damp_results_g{gamma_val:.2f}.npz") 
results_wf_ld = None
if os.path.exists(results_file):
    try:
        data = np.load(results_file, allow_pickle=True)
        results_wf_ld = {key: data[key].item() if data[key].ndim == 0 else data[key] for key in data.files}
        print(f"Loaded data for n'={n_prime_bg:.1f} (gamma={gamma_val:.2f}) from {results_file}")
    except Exception as e:
        print(f"Error loading file {results_file}: {e}")
else:
    print(f"Result file not found: {results_file}")

if results_wf_ld:
    # Plot Stability Metrics vs Time 
    plt.figure(figsize=(18, 5))
    plot_steps = results_wf_ld['steps_completed'] 
    num_data_points = len(results_wf_ld['max_amp'])
    steps_axis = np.linspace(0, plot_steps, num_data_points) # Create axis based on actual points
    
    label_str = f"T/S (n'={n_prime_bg:.1f}, γ={gamma_val:.2f})"
    linestyle = '-' if results_wf_ld['stable'] else ':'
    if not results_wf_ld['stable']:
         label_str += " (Unstable)"
             
    # Max Amplitude
    ax1 = plt.subplot(1, 3, 1)
    ax1.plot(steps_axis, results_wf_ld['max_amp'], label=label_str, linestyle=linestyle)
    ax1.set_xlabel(f"Step")
    ax1.set_ylabel("Max |φ|")
    ax1.set_title("Max Amplitude vs Time (T/S, Lin. Damp.)")
    ax1.grid(True, which="both", ls="--")
    ax1.legend(fontsize='small')

    # Average Density
    ax2 = plt.subplot(1, 3, 2)
    target_rho_wf = rho_ref / n_prime_bg
    ax2.plot(steps_axis, results_wf_ld['avg_rho'], label=label_str, linestyle=linestyle)
    ax2.axhline(target_rho_wf, linestyle='--', color='gray', alpha=0.5, label='Target <ρ>')
    ax2.set_xlabel(f"Step")
    ax2.set_ylabel("Average Density <ρ>")
    ax2.set_title("Average Density vs Time (T/S, Lin. Damp.)")
    ax2.grid(True, which="both", ls="--")
    ax2.legend(fontsize='small')

    # Total Energy
    ax3 = plt.subplot(1, 3, 3)
    ax3.plot(steps_axis, results_wf_ld['energy'], label=label_str, linestyle=linestyle)
    ax3.set_xlabel(f"Step")
    ax3.set_ylabel("Total Energy E")
    ax3.set_title("Total Energy vs Time (T/S, Lin. Damp.)")
    ax3.grid(True, which="both", ls="--")
    ax3.legend(fontsize='small')

    plt.tight_layout()
    plt.savefig(os.path.join(plot_path, f"wf_analogue_lin_damp_N{N}_stability_metrics.png"))
    plt.show()

    # Visualize Final State if stable
    if results_wf_ld['stable'] and results_wf_ld['final_phi_slice'] is not None and results_wf_ld['final_phi_slice'].size > 0:
        plt.figure(figsize=(6, 5))
        slice_data = results_wf_ld['final_phi_slice'].astype(np.float32)
        if np.isnan(slice_data).any():
             print("Warning: NaN found in final slice, skipping visualization.")
        else:
            im = plt.imshow(slice_data, extent=[-L/2, L/2, -L/2, L/2], cmap='viridis', aspect='auto')
            plt.colorbar(im)
            plt.title(f"Final φ (z=0) for T/S State (n'={n_prime_bg:.1f}, γ={gamma_val:.2f})")
            plt.xlabel("x")
            plt.ylabel("y")
            plt.savefig(os.path.join(plot_path, f"wf_analogue_lin_damp_N{N}_final_state.png"))
            plt.show()
    elif not results_wf_ld['stable']:
        print("Simulation was unstable, final state not visualized.")
    else:
         print("No final state data available for visualization.")
else:
    print("No results loaded to analyze.")

## 6. Simulation Report (T/S with Linear Damping)

In [ ]:
# --- Final Report Summary for T/S with Linear Damping ---
print("\n--- T/S Simulation Report (Linear Damping) ---")
print(f"Simulation Timestamp: {datetime.now()}")
print(f"Grid Size (N): {N}")
print(f"Linear Damping (γ): {gamma_damp}")

if results_wf_ld:
    print("Stability Summary:")
    status = "Stable" if results_wf_ld['stable'] else "Unstable"
    steps_comp = results_wf_ld.get('steps_completed', T_steps)
    final_amp_str = f"{results_wf_ld['max_amp'][-1]:.2f}" if results_wf_ld['stable'] and results_wf_ld['max_amp'].size > 0 else "N/A"
    final_rho_str = f"{results_wf_ld['avg_rho'][-1]:.4f}" if results_wf_ld['stable'] and results_wf_ld['avg_rho'].size > 0 else "N/A"
    final_energy_str = f"{results_wf_ld['energy'][-1]:.2e}" if results_wf_ld['stable'] and results_wf_ld['energy'].size > 0 else "N/A"
    print(f" Result for n'={results_wf_ld['n_prime']:.1f}: {status} (Completed: {steps_comp}/{T_steps}, Final Max|φ|={final_amp_str}, Final <ρ>={final_rho_str}, Final E={final_energy_str})")
else:
    print("No simulation results loaded to report.")

print("\nAnalysis plots potentially saved to:", plot_path)
print("Data files potentially saved to:", data_path)
print("Checkpoints potentially saved to:", checkpoint_path)
print("-------------------------------------------")